In [15]:
from davinci_still_tool.resolve_init import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

media_pool_item = root_folder.GetSubFolderList()[0].GetClipList()[0]
media_pool.AppendToTimeline(
    [{"mediaPoolItem": media_pool_item, "startFrame": 1, "endFrame": 1}]
)
current_timeline.GetItemListInTrack("video", 1)[0].GetMarkers()

timelineItems = current_timeline.GetItemListInTrack("video", 1)
for timelineItem in timelineItems:
    timelineItemStartFrame = timelineItem.GetStart()
    timelineItemLeftOffset = timelineItem.GetLeftOffset()
    markers = timelineItem.GetMarkers()
    for markerStartFrame in markers.keys():
        print(timelineItemStartFrame - timelineItemLeftOffset + markerStartFrame)
        print(timelineItemStartFrame, timelineItemLeftOffset, markerStartFrame)
        break
    break

86720
86792 127 55


In [22]:
def append_to_timeline(self) -> None:
    """Append to timeline"""
    all_timeline_name = [
        timeline.GetName() for timeline in self.get_all_timeline()  # type: ignore
    ]
    for subfolder in self.root_folder.GetSubFolderList():
        for clip in subfolder.GetClipList():
            if (
                clip.GetClipProperty("type") == "Video"
                or clip.GetClipProperty("type") == "Video + Audio"
            ):
                clip_width = clip.GetClipProperty("Resolution").split("x")[0]
                clip_height = clip.GetClipProperty("Resolution").split("x")[1]
                for name in all_timeline_name:
                    if f"{clip_width}x{clip_height}" in name:
                        self.project.SetCurrentTimeline(
                            self.get_timeline_by_name(name)
                        )
                        self.media_pool.AppendToTimeline(clip)